In [2]:
import pandas as pd
from tabulate import tabulate

def cal_range(no_of_units):
    ranges = []
    n = no_of_units // 25
    for k in range(25):
        initial = 1 + k * n
        final = min((k + 1) * n, no_of_units)
        quantity_range = f"{initial}-{final}"
        ranges.append((initial, final))
    return ranges

def calculate_total_cost(F, V, Q):
    total_cost = F + V * Q
    return total_cost

def calculate_profit_or_loss(total_revenue, total_cost):
    profit_or_loss = total_revenue - total_cost
    return profit_or_loss

def calculate_profit_or_loss_per_unit(unit_price, unit_cost):
    profit_or_loss_per_unit = unit_price - unit_cost
    return profit_or_loss_per_unit

def calculate_breakeven_quantity(F, P, V):
    if P > V:
        breakeven_quantity = F / (P - V)
        return breakeven_quantity
    else:
        return None  # Return None if breakeven quantity is not defined

# Collect inputs from the user with error handling
while True:
    try:
        product_name = input("Enter the product name: ")
        F = float(input("Enter the fixed operating costs: "))
        V = float(input("Enter the variable cost per unit: "))
        P = float(input("Enter the sales price per unit: "))
        N = int(input("Enter number of units: "))

        if F < 0 or V < 0 or P < 0 or N < 0:
            raise ValueError("Inputs cannot be negative.")
        
        break
    except ValueError as e:
        print("Invalid input. Please enter valid numeric values.")

# Create ranges
ranges = cal_range(N)

# Initialize lists to store results
total_costs = []
total_revenues = []
profits_or_losses = []
profits_or_losses_per_unit = []

# Calculate values for each range
for initial, final in ranges:
    Q = (initial + final) // 2  # Use the midpoint of the range for calculations
    total_revenue = P * Q
    total_cost = calculate_total_cost(F, V, Q)
    profit_or_loss = calculate_profit_or_loss(total_revenue, total_cost)
    profit_or_loss_per_unit = calculate_profit_or_loss_per_unit(P, V)
    
    total_costs.append(total_cost)
    total_revenues.append(total_revenue)
    profits_or_losses.append(profit_or_loss)
    profits_or_losses_per_unit.append(profit_or_loss_per_unit)

# Calculate break-even quantity
break_even_quantity = calculate_breakeven_quantity(F, P, V)

# Create a DataFrame with the results
data = {
    'Range': [f"{initial}-{final}" for initial, final in ranges],
    'Total Cost': total_costs,
    'Total Revenue': total_revenues,
    'Profit or Loss': profits_or_losses,
    'Profit or Loss per Unit': profits_or_losses_per_unit,
    'Breakeven Quantity': [break_even_quantity] * 25
}
df = pd.DataFrame(data)

# Find row(s) closest to breakeven point
if break_even_quantity is not None:
    closest_index = min(range(len(ranges)), key=lambda i: abs((ranges[i][0] + ranges[i][1]) // 2 - break_even_quantity))
    df.at[closest_index, '*'] = '*'  # Mark the closest row with a star
else:
    closest_index = None

# Determine if the product is profitable
is_profitable = sum(profits_or_losses) > 0

# Open the file for appending
with open('output.txt', 'a') as f:
    f.write(f"\nProduct Name: {product_name}\n")
    f.write(f"Fixed Operating Costs: ${F}\n")
    f.write(f"Variable Cost per Unit: ${V}\n")
    f.write(f"Sales Price per Unit: ${P}\n")
    f.write(f"Number of Units: {N}\n\n")
    f.write(f"{'-' * 60}\n")
    f.write(f"{' ' * 15}{product_name}\n")
    f.write(f"{'-' * 60}\n")
    f.write(tabulate(df, headers='keys', tablefmt='grid', showindex=False))
    f.write("\n\n")
    f.write(f"{'*' * 60}\n")
    f.write("\n")
    f.write(f"Profitable: {'Yes' if is_profitable else 'No'}\n")
    if break_even_quantity is not None:
        f.write(f"Breakeven Quantity: {break_even_quantity}\n")
    else:
        f.write("Breakeven Quantity: Not applicable\n")

print("Output has been appended to 'output.txt'.")
